In [2]:
import sys
assert sys.version_info >= (3, 5)
import sklearn
assert sklearn.__version__ >= "0.20"
import tensorflow as tf
import string
import re
assert tf.__version__ >= "2.0"
import numpy as np
import os
import keras
import random
import subprocess
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction
np.random.seed(42)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")
#print(subprocess.check_output("nvidia-smi --query-gpu=name --format=csv").decode("ascii"))
#print(subprocess.check_output("nvidia-smi --query-gpu=memory.total --format=csv").decode("ascii"))

In [3]:

text_file = "C:\\Users\\Emanuel\\Desktop\\AD\\TP2\\WikiMatrix.es-pt.tsv"
with open(text_file, encoding="utf8") as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    _, es, pt = line.split("\t")
    pt = "[start] " + pt + " [end]"
    text_pairs.append((es, pt))
del lines
del _
print(f"Tamanho do Dataset : {len(text_pairs)}")
text_pairs = random.sample(text_pairs, 100000)
print(f"Tamanho usado : {len(text_pairs)}")

Tamanho do Dataset : 2251342
Tamanho usado : 100000


In [4]:
for x in range(3):
  print(x+1 , ": ", random.choice(text_pairs))

1 :  ('Centro de Guerra de la Fuerza Aérea de Estados Unidos (USAFWC), Base de la Fuerza Aérea Nellis, Nevada.', '[start] A Força Aérea dos Estados Unidos atualmente tem uma escola similar, chamada de United States Air Force Fighter Weapons School em Nellis Air Force Base, Nevada. [end]')
2 :  ('«¿Por qué se hizo?».', '[start] Por que ele fez aquilo?" [end]')
3 :  ('En caso de accidente o malestar, se deberá acudir inmediatamente al médico (si es posible, mostrarle la etiqueta).', '[start] Se o gatilho for conhecido, deve ser removido se possível (por exemplo, a causa da dor). [end]')


In [6]:
# Dividir o datastet em 3 conjuntos: treino, validação e teste
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [7]:
from tensorflow.keras import layers

strip_chars = string.punctuation
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")
strip_chars += "«»—–ºª"



def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    lowercase = tf.strings.regex_replace(lowercase, "\\xa0", "")
    lowercase = tf.strings.regex_replace(lowercase, "\\[\w\d]+", "")
    return tf.strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
    standardize=custom_standardization
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization
)
train_es_texts = [pair[0] for pair in train_pairs]
train_pt_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_es_texts)
target_vectorization.adapt(train_pt_texts)

In [ ]:
#source_vectorization.get_vocabulary()
#target_vectorization.get_vocabulary()

In [7]:
batch_size = 64

def format_dataset(es, pt):
    es = source_vectorization(es)
    pt = target_vectorization(pt)
    return ({
        "spanish": es,
        "portuguese": pt[:, :-1],
    }, pt[:, 1:])

def make_dataset(pairs):
    es_texts, pt_texts = zip(*pairs)
    es_texts = list(es_texts)
    pt_texts = list(pt_texts)
    dataset = tf.data.Dataset.from_tensor_slices((es_texts, pt_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [8]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [9]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [10]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [11]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="portuguese")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)

transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [12]:
transformer.compile(
    optimizer="SGD",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
1094/1094 [==============================] - 48s 41ms/step - loss: 5.4226 - accuracy: 0.1467 - val_loss: 5.0131 - val_accuracy: 0.1687
Epoch 2/10
1094/1094 [==============================] - 45s 41ms/step - loss: 4.9857 - accuracy: 0.1760 - val_loss: 4.8270 - val_accuracy: 0.1913
Epoch 3/10
1094/1094 [==============================] - 44s 40ms/step - loss: 4.8356 - accuracy: 0.1916 - val_loss: 4.7080 - val_accuracy: 0.2050
Epoch 4/10
1094/1094 [==============================] - 44s 40ms/step - loss: 4.7376 - accuracy: 0.2024 - val_loss: 4.6275 - val_accuracy: 0.2123
Epoch 5/10
1094/1094 [==============================] - 44s 40ms/step - loss: 4.6670 - accuracy: 0.2083 - val_loss: 4.5658 - val_accuracy: 0.2174
Epoch 6/10
1094/1094 [==============================] - 45s 41ms/step - loss: 4.6107 - accuracy: 0.2128 - val_loss: 4.5153 - val_accuracy: 0.2217
Epoch 7/10
1094/1094 [==============================] - 44s 41ms/step - loss: 4.5625 - accuracy: 0.2171 - val_loss: 4.4712 -

In [13]:
# Testar o desempenho do Transformer em frases do conjunto de teste

import numpy as np
pt_vocab = target_vectorization.get_vocabulary()
pt_index_lookup = dict(zip(range(len(pt_vocab)), pt_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = pt_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

def decode_sequence_wout_startend(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = pt_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence.replace("[start]", "").replace("[end]", "")

bleu_pt_list = []
bleu_translation_list = []

for _ in range(100):
    pair = random.choice(test_pairs)
    # Dividir o pares em listas de palavras, meter tudo em lower case e retirar start/end
    es_text = pair[0].lower().split(" ")
    pt_text = pair[1].replace("[start]", "").replace("[end]", "").lower().split(" ")
    
    while len(es_text) <= 4 or len(pt_text) <= 4:
        pair = random.choice(test_pairs)
        es_text = pair[0].lower().split(" ")
        pt_text = pair[1].replace("[start]", "").replace("[end]", "").lower().split(" ")
        
    # Traduzir a frase
    translation = decode_sequence_wout_startend(pair[0]).lower().split(" ")
    
    # Filtrar strings vazias
    pt_text = list(filter(None, pt_text))
    translation = list(filter(None, translation))
    bleu_pt_list.append([pt_text])
    bleu_translation_list.append(translation)

print(bleu_pt_list[0])
print(bleu_translation_list[0])

# Calcular cumullative 4-gram BLEU
gram4 = corpus_bleu(bleu_pt_list, bleu_translation_list, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=SmoothingFunction().method1)
print(f'4-gram BLEU: {round(np.mean(gram4),3)}')

[['the', 'sword', 'of', 'hajya', 'é', 'uma', 'continuação', 'direta', 'da', 'história', 'de', 'shining', 'force', 'gaiden;', 'este', 'jogo', 'se', 'passa', 'apenas', 'dois', 'meses', 'depois', 'e', 'cobre', 'a', 'resolução', 'da', 'guerra', 'entre', 'as', 'forças', 'de', 'cypress', 'e', 'os', 'seguidores', 'de', 'iom.']]
['em', '[unk]', '[unk]', '[unk]', '[unk]', '[unk]', '[unk]', '[unk]', '[unk]', '[unk]', '[unk]', '[unk]', '[unk]', '[unk]', '[unk]', '[unk]']
4-gram BLEU: 0.001


In [14]:
test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(10):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Ahora mi vida ha cambiado, sin razón aparente.
[start] [UNK] [UNK] [UNK] [end]
-
El mazo puede contener un máximo de cuatro cartas de oro diferentes y seis cartas de plata diferentes.
[start] o [UNK] [UNK] [UNK] um [UNK] de [UNK] [UNK] de [UNK] [end]
-
Esta agrupación, después de firmar un contrato con Universal Music, se cambió el nombre a «Tokio Hotel».
[start] a [UNK] foi um [UNK] de [UNK] de [UNK] [UNK] [UNK] [end]
-
El verdadero nombre de Angel es Jennifer Kita.
[start] o [UNK] [UNK] de [UNK] [end]
-
MacGregor ha desempeñado una variedad de papeles en la televisión desde el 2000.
[start] [UNK] [UNK] [UNK] um [UNK] de [UNK] em inglês [end]
-
Esto marcó un hito, ya que fue la cantidad más alta que se había pagado hasta entonces por derechos de autor.
[start] o [UNK] um [UNK] um [UNK] que o [UNK] que se [UNK] que se [UNK] que o [UNK] [end]
-
La vereda posee una belleza natural única.
[start] a [UNK] [UNK] [UNK] [UNK] [UNK] [end]
-
Después de que Indonesia obtuviese su independencia